**Company** : <br>
Staffing Firm

**Notebook Function** : <br>
    This notebook walks through the steps to finetune mittens by starting with the initial company embeddings that were trained from the Staffing firm corpus

**Output File(s)** : <br>
     - A folder containing the user embeddings for each time period

**Author(s)** : <br>
Lara Yang, Sarayu Anshuman

Import libraries

In [ ]:
import os
import sys
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
from mittens import Mittens
import csv
from operator import itemgetter
import ujson as json
import re
from gensim.matutils import cossim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from statistics import mean 
from sklearn.decomposition import PCA

Run the helper functions

In [ ]:
#########################################################################
########### Helper Functions for Generating Mittens Embeddings ##########
#########################################################################

#thsi function returns a word, another word, and the value given in the cooccurance matrix based on the weighting function of the occurance of teh second word in the first word's context window
def _window_based_iterator(toks, window_size, weighting_function):
    for i, w in enumerate(toks):
        yield w, w, 1
        left = max([0, i-window_size])
        for x in range(left, i):
            yield w, toks[x],weighting_function(x)
        right = min([i+1+window_size, len(toks)])
        for x in range(i+1, right):
            yield w, toks[x], weighting_function(x)
    return

def glove2dict(glove_filename):
    """
    Reads word vectors into a dictionary
    Parameters
    ----------
    glove_filename : str
        Name of file that contains vectors
    Returns
    -------
    data : dict
        A dictionary matching words to their vectors
    """
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE) #GloVe output files have the format one word per line, followed by its vector values separated by spaces
        data = {line[0]: np.array(list(map(float, line[1: ]))) for line in reader} #create a key (word): value (embedding) dict for every word
    return data

# Inspired by original build_weighted_matrix in utils.py in the Mittens paper source codebase
def build_weighted_matrix(emails,
        mincount=300, vocab_size=None, window_size=10,
        weighting_function=lambda x: 1 / (x + 1),
        email_type='all'):
    """
    Builds a count matrix based on a co-occurrence window of
    `window_size` elements before and `window_size` elements after the
    focal word, where the counts are weighted based on proximity to the
    focal word.
    Parameters
    ----------
    emails : list of dicts
        Emails converted from JSON formats
    mincount : int
        Only words with at least this many tokens will be included. #this means only words with atleast 300 occurances in teh document are included
    vocab_size : int or None
        If this is an int above 0, then, the top `vocab_size` words
        by frequency are included in the matrix, and `mincount`
        is ignored.
    window_size : int
        Size of the window before and after. (So the total window size
        is 2 times this value, with the focal word at the center.)
    weighting_function : function from ints to floats
        How to weight counts based on distance. The default is 1/d
        where d is the distance in words.
    email_type : str, optional
        Specifies which types of emails to include when building embeddings
    Returns
    -------
    X : pd.DataFrame
        Cooccurence matrix guaranteed to be symmetric because of the way the counts are collected.
    """
    wc = defaultdict(int)
    #corpus contains a list  of emails, where each email is a dict
    corpus = read_corpus(emails, email_type=email_type, sentence_delim=False)
    if corpus is None:
        print("These emails are empty\t{}.\nEmpty corpus returned for email type {}".format(str(emails), email_type))
        return pd.DataFrame()
    for toks in corpus:
        for tok in toks:
            wc[tok] += 1 #word count
    #now create the vocabulary
    if vocab_size: #if a vocab size is defined then take the first top 'vocab_size' counts
        srt = sorted(wc.items(), key=itemgetter(1), reverse=True)
        vocab_set = {w for w, c in srt[: vocab_size]} #sort all the words in the vocabulary according to count
    else: #define a vocab based on all those words which have a count greater than 'mincount'
        vocab_set = {w for w, c in wc.items() if c >= mincount}
    vocab = sorted(vocab_set)
    n_words = len(vocab) #length of vocab
    # Weighted counts:
    counts = defaultdict(float)
    for toks in corpus: #for each email
        window_iter = _window_based_iterator(toks, window_size, weighting_function)
        for w, w_c, val in window_iter:
            if w in vocab_set and w_c in vocab_set:
                counts[(w, w_c)] += val

    '''
    For each sentence (list of words), it uses a helper function _window_based_iterator (not provided) to generate a sequence of tuples:
    The first element is the focal word.
    The second element is a co-occurring word within the window.
    The third element is the weight for the co-occurrence based on the distance between the words (using the provided weighting_function).
    It checks if both the focal word and co-occurring word are in the vocabulary (vocab_set). If so, it updates the counts dictionary with the weighted co-occurrence for that word pair.
    '''
    X = np.zeros((n_words, n_words))
    for i, w1 in enumerate(vocab):
        for j, w2 in enumerate(vocab):
            X[i, j] = counts[(w1, w2)]
    X = pd.DataFrame(X, columns=vocab, index=pd.Index(vocab))
    return X

def read_corpus(emails, email_type, sentence_delim=False):
    """
    Parameters
    ----------
    emails : list of dict
        A list of emails converted from JSON formats
    email_type : str
        Specifies which types of emails to include when building embeddings
        'internal' filters for internal emails, 'external' filters for external and mixed emails, 
        and anything else does not filter
    sentence_delim : bool, optional
        If true, co-occurrences across sentence boundaries are ignored.
    Returns
    -------
    list of list of str
        Corpus converted from emails.
        If sentence_delim is false, returns a list of emails, which are represented as lists of tokens
        If sentence_delim is true, returns a list of sentences, which are represented as lists of tokens
    """
    # split with no argument splits on whitespaces and newlines
    if not sentence_delim:
        if email_type == 'internal':
            return [email['hb'].replace('\n', ' ').replace("SENT_END", "").strip().split() for email in emails if (
                email['email_type'] == 'int')]
        elif email_type == 'external':
            return [email['hb'].replace('\n', ' ').replace("SENT_END", "").strip().split() for email in emails if (
                email['email_type'] == 'ext' or email['email_type'] == 'mixed')]
        # agnostic to email type
        else:
            return [email['hb'].replace('\n', ' ').replace("SENT_END", "").strip().split() for email in emails]
    else:
        if email_type == 'internal':
            return [sent.strip().split() for email in emails for line in email['hb'].split('\n') for sent in line.split('SENT_END') if (
                len(sent) > 0 and email_type == 'int')]
        elif email_type == 'external':
            return [sent.strip().split() for email in emails for line in email['hb'].split('\n') for sent in line.split('SENT_END') if (
                len(sent) > 0 and (email_type == 'int' or email['email_type'] == 'mixed'))]
        else:
            return [sent.strip().split() for email in emails for line in email['hb'].split('\n') for sent in line.split('SENT_END') if len(sent) > 0]

def output_embeddings(mittens_df, filename, compress=False):
    if compress:
        mittens_df.to_csv(filename + '.gz', quoting=csv.QUOTE_NONE, header=False, sep=" ", compression='gzip')
    else:
        mittens_df.to_csv(filename, quoting=csv.QUOTE_NONE, header=False, sep=" ")
    return

#########################################################################
############# Helper Functions for Working with JSON Emails #############
#########################################################################
# Slightly different from get_recipients for spacespace emails
def get_recipients(msg):
    """
    Return a set of recipients of the current message.
    self is removed from list of recipients if in recipients #(-set([sender]))
    All fields contain email addresses, not user IDs. From fields are visually just strings
    but checking just in case
    """
    sender = msg['From'][0] if type(msg['From']) == list else msg['From']
    return set(msg.get('To', []) + msg.get('Cc', []) + msg.get('Bcc', [])) - set([sender]) #basically return a set of recipients to the current email

def slice_user_corpus(emails, train_mode): #slices a list of emails into chunks based on time periods specified by the train_mode parameter
    """
    Parameters
    ----------
    emails : list of dict
        A list of emails converted from JSON formats to dictionaries 
    train_mode : str
        One of 'annual', 'quarterly', 'all'
        Indicates how to chunk up emails - into quarters, years, or both
    Returns
    -------
    timekey2emails : dict
        Matches quarters or years to respective emails
    """
    #'ActivityCreatedAt' contains a timestamp for the list of emails
    timekey2emails = defaultdict(list)
    #the function iterates through each email in the email list
    for email in emails:
        if train_mode == 'annual':
            timekey2emails[to_year(email['ActivityCreatedAt'], format='str')].append(email)
        elif train_mode == 'quarterly':
            timekey2emails[to_quarter(email['ActivityCreatedAt'], format='str')].append(email)
        elif train_mode == 'halfyear':
            timekey2emails[to_halfyear(email['ActivityCreatedAt'], format='str')].append(email)
        elif train_mode == 'all':
            timekey2emails[to_year(email['ActivityCreatedAt'], format='str')].append(email)
            timekey2emails[to_quarter(email['ActivityCreatedAt'], format='str')].append(email)
            timekey2emails[to_halfyear(email['ActivityCreatedAt'], format='str')].append(email)
    return timekey2emails

#########################################################################
############# Helper Functions for Working with Date Objects ############
#########################################################################

def to_quarter(date, format):
    """
    Return quarter of date in string
    """
    year, month = 0, 0
    if format == 'str':
        year = date[0:4]
        month = date[5:7]    
    elif format == 'datetime':
        year = date.year
        month = date.month
    quarter = ((int(month)-1) // 3) + 1
    timekey = str(year) + 'Q' + str(quarter)
    return timekey

def to_halfyear(date, format):
    """
    Return half year of date in string
    """
    year, month = 0, 0
    if format == 'str':
        year = date[0:4]
        month = date[5:7]    
    elif format == 'datetime':
        year = date.year
        month = date.month
    halfyear = ((int(month)-1) // 6) + 1
    timekey = str(year) + 'HY' + str(halfyear)
    return timekey

def to_year(date, format):
    """
    Return year of date in string
    """
    if format == 'str':
        return date[0:4]
    elif format == 'datetime':
        return str(date.year)

def datetime_to_timekey(date, time_key):
    if time_key == 'year':
        return to_year(date, format='datetime')
    elif time_key == 'quarter':
        return to_quarter(date, format='datetime')

def is_month_before_equal(datetime1, datetime2):
    if datetime1.year < datetime2.year:
        return 1
    elif (datetime1.year == datetime2.year) and (datetime1.month <= datetime2.month):
        return 1
    else:
        return 0

def num_months_between_dates(datetime1, datetime2):
    return abs((datetime1.year - datetime2.year) * 12 + datetime1.month - datetime2.month)

def num_quarters_between_dates(datetime1, datetime2):
    return abs((datetime1.year - datetime2.year) * 12 + datetime1.month - datetime2.month) // 3

def num_years_between_dates(datetime1, datetime2):
    return abs(datetime1.year - datetime2.year)

def time_between_dates(datetime1, datetime2, time_key):
    if time_key == 'monthly':
        return num_months_between_dates(datetime1, datetime2)
    elif time_key == 'quarterly':
        return num_quarters_between_dates(datetime1, datetime2)
    elif time_key == 'annual':
        return num_years_between_dates(datetime1, datetime2)

#########################################################################
############## Helper Functions for Working with Dataframes #############
#########################################################################

#function to convert a dictionary to a dataframe
def dict_to_df(index2rows, cols, index_name):
    """
    Parameters
    ----------
    index2rows : dict
        Dictionary mapping index to rows to be coverted
    cols : list
        List of column names of type str
    index : list
        List of index names
    Returns
    -------
    df : pd.DataFrame
        Constructed dataframe
    """
    if index2rows is None or len(index2rows) == 0:
        return None
    #case where the dataframe is formed across entire time span of user emails
    if len(index_name) == 1:
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df.index.name = index_name[0]
        df.sort_index(axis=0, inplace=True)
        return df
    else: #case where the user token (word) counts and email counts are formed for ecah year or each quarter
        df = pd.DataFrame.from_dict(index2rows, orient='index', columns=cols)
        df = pd.DataFrame(df, pd.MultiIndex.from_tuples(df.index, names=index_name))
        df.sort_index(axis=0, inplace=True)
        return df

#########################################################################
########### Helper Functions for Working with Embedding Output ##########
#########################################################################

def remove_empty_embeddings(embeddings_dir):
    """
    Removes all empty files in embeddings_dir that were produced when vocab size was 0.
    Parameters
    ----------
    embeddings_dir : str
        Full path to directory where embedding files are located
    """
    for file in os.listdir(embeddings_dir):
        mittens_file = os.path.join(embeddings_dir, file)
        if os.path.getsize(mittens_file) == 0:
            os.remove(mittens_file)
    return

#not called, because this functionm is needed for finetuning and not training
def extract_company_embedding(company_embeddings_filename, tmp_dir, words):
    """
    Parameters
    ----------
    company_embeddings_filename : str
        File path of the company embeddings
    tmp_dir : str
        Path to the directory for gensim to output its tmp files in order to load embeddings into word2vec format
    words : list
        A list of strings for which to retrieve vectors
    Returns
    -------
    vecs : list
        A list of vectors of type numpy.ndarray that correspond to the list of words given as parameters
    """ 
    tmp_mittens = os.path.join(tmp_dir, "mittens_embeddings_all_word2vec.txt")
    word2vec_mittens_file = get_tmpfile(tmp_mittens)
    glove2word2vec(company_embeddings_filename, word2vec_mittens_file) #load the embeddings in the glove2word2vec format
    model = KeyedVectors.load_word2vec_format(word2vec_mittens_file) #this model represents the glove embddings space using the default company embeddings
    vecs = []
    for w in words:
        if w in model.vocab:
            vecs.append(model.wv[w]) #default vectors for the words in the vocab
        else:
            vecs.append(np.nan)
            print('%s not in company embeddings' % w)
    return vecs

def word_similarity(model, w1, w2):
    """
    This is an auxilary function that allows for comparing one word to another word or multiple words
    If w1 and w2 are both single words, n_similarity returns their cosine similarity which is the same as 
    simply calling similarity(w1, w2)
    If w1 or w2 is a set of words, n_similarity essentially takes the mean of the set of words and then computes
    the cosine similarity between that vector mean and the other vector. This functionality is both reflected
    in its source code and has been verified manually.
    Parameters
    ----------
    model : KeyedVectors
        The model that contains all the words and vectors
    w1 : str or list
        The first word or word list to be compared
    w2 : str or list
        The second word or word list to be compared
    Returns
    -------
    float
        Cosine similarity between w1 and w2
    """
    #first fo the case where w1 and w2 parameters are single words
    if not isinstance(w1, list): 
        w1 = [w1]
    if not isinstance(w2, list):
        w2 = [w2]
    #create a copy of the input lists
    w1 = [w for w in w1 if w in model.vocab]
    w2 = [w for w in w2 if w in model.vocab]
    if len(w1) == 0 or len(w2) == 0:
        return None
    return model.n_similarity(w1, w2) #inbuilt word2vec model that calculates the cosine similarity between two lists; takes the mean of the set of words and then computes
    #the cosine similarity between that vector mean and the other vector

def cossim_with_none(vec1, vec2, vec_format='sparse'):
    """
    Auxiliary function that calls cossim function to test if vectors are None to prevent erroring out.
    Parameters
    ----------
    vec1 : list of (int, float), gensim sparse vector format
    vec2 : list of (int, float), gensim sparse vector format
    format : str, optional
        Either sparse or dense. If sparse, vec1 and vec2 are in gensim sparse vector format; use cossim function from gensim.
        Otherwise, vec1 and vec2 are numpy arrays and cosine similarity is hand calculated
    Returns
    -------
    float
        Cosine similarity between vec1 and vec2
    """
    if not (isnull_wrapper(vec1) or isnull_wrapper(vec2)):
        if vec_format == 'sparse':
            return cossim(vec1, vec2)
        elif vec_format == 'dense':
            if len(vec1) == 0 or len(vec2) == 0:
                return None
            return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))
        else:
            raise ValueError()
    return None

def calculate_pairwise_cossim(col1, col2=None, reference=False, reference_group=None, anon_ids=None, vec_format='sparse'):
    """
    Calculates averaged cosine similarity of every row vector in col1 with every other row vector in col2.
    If no col2 is provided, cosine similarity of every row vector with every other row vector in col1 is calculated.
    The two columns should have equal length.
    Parameters
    ----------
    col1 : pd.Series
        A column where each row is a sparse word vector (BoW format that gensim code is written for).
    col2 : pd.Series, optional
        A column where each row is a sparse word vector (BoW format that gensim code is written for).
    reference : bool, optional
        Indicator variable for whether filtering for reference groups is needed
    reference_group : pd.Series, optional
        If filtering for reference groups, a list containing reference group members for every employee in col1
    anon_ids : pd.Series, optional
        If filtering for reference groups, a list containing anon_ids for every employee in col1
    Returns
    -------
    results : list
        A list where the ith element is the averaged cosine similarity between the ith vector in col1 and every vector
        in col2 for which i != j. If no col2 is provided, a list where the ith element is the averaged cosine similarity
        between the ith vector in col1 and every other vector in col1 is returned.
    """
    vectors1 = col1.tolist()
    vectors2 = col2.tolist() if col2 is not None else col1.tolist()
    reference_group = reference_group.tolist() if reference else None
    anon_ids = anon_ids.tolist() if anon_ids is not None else None
    results = list()
    for i in range(len(vectors1)):
        total_sim = []
        if not isnull_wrapper(vectors1[i]):
            for j in range(len(vectors2)):
                if i != j and not isnull_wrapper(vectors2[j]):
                    # filter out any np.nans as our reference group
                    if not reference or (type(reference_group[i]) == set and anon_ids[j] in reference_group[i]):
                        if vec_format == 'sparse':
                            total_sim.append(cossim(vectors1[i], vectors2[j]))
                        elif vec_format == 'dense':
                            total_sim.append(np.dot(vectors1[i], vectors2[j])/(np.linalg.norm(vectors1[i]) * np.linalg.norm(vectors2[j])))
                        else:
                            raise ValueError()
            results.append(mean(total_sim) if len(total_sim) > 0 else None)
        else:
            results.append(None)
    return results

def isnull_wrapper(x):
    r = pd.isnull(x)
    if type(r) == bool:
        return r
    return r.any()

def vector_mean(col):
    """
    Calculate vector means of row vectors
    Parameters
    ----------
    col : pd.Series
        The column to be averaged
    Returns
    -------
    np.array
        A vector that is the numerical average of all vectors in col
    """
    return np.array(col[col.notna()].tolist()).mean(axis=0)

def extract_hr_survey_df(survey_hr_file, user_qualtrics_file, users_file, perf_likert_file, perf_percentage_file):
    """
    Loads various datasets to prepare survey and hr data for merging with embeddings df.
    Returns
    -------
    survey_hr_df : pd.DataFrame
        Dataframe indexed by user_id
    """
    perf_likert_df = pd.read_csv(perf_likert_file)
    perf_percentage_df = pd.read_csv(perf_percentage_file)
    perf_likert_df = perf_likert_df[['UID', '2019 Perf_Type(Rating)', '2020 Perf_Type(Rating)']]
    perf_percentage_df = perf_percentage_df[['UID', '2019_Perf_Type(Percentage)', '2020_Perf_Type(Percentage)']]
    perf_likert_df.columns = ["UID", "perf_rating_2019", "perf_rating_2020"]
    perf_percentage_df.columns = ["UID", "perf_percentage_2019", "perf_percentage_2020"]
    
    user_qualtrics_df = pd.read_csv(user_qualtrics_file)
    user_qualtrics_df = user_qualtrics_df.merge(perf_likert_df, on='UID', how='left').merge(perf_percentage_df, on="UID", how='left')

    survey_hr_df = pd.read_csv(survey_hr_file)
    survey_hr_df = survey_hr_df.merge(user_qualtrics_df, left_on='uid', right_on='UID', how='left')
    # we lose two employees whose emails are not included in the crawled email data
    email2uid = {}
    with open(users_file, encoding='utf-8') as f:
        for line in f:
            user = json.loads(line)
            for e in user['Emails']:
                email2uid[e] = user['UserId']

    survey_hr_df = survey_hr_df[survey_hr_df['Email'].isin(email2uid.keys())]
    survey_hr_df['user_id'] = survey_hr_df['Email'].apply(lambda e : email2uid[e])
    survey_hr_df.set_index('user_id', inplace=True)
    return survey_hr_df

def extract_variables_from_file(file):
    """ 
    Extract relevant information from name of embedding file, with format: {}(_{})?_(internal|external).txt
    """
    file_chunks = file[0:-4].split('_')
    usr = file_chunks[0]
    time_key = file_chunks[1] if len(file_chunks) == 3 else None
    return (usr, time_key)

def project(word, dimension):
    """
    Returns the scalar projection of word on dimension. Word and dimension are both assumed to be vectors
    """
    return np.dot(word, dimension)/ np.linalg.norm(dimension)

def drop(u, v):
    """
    Returns the component in u that is orthogonal to v, also known as the vector rejection of u from v
    """
    return u - v * u.dot(v) / v.dot(v)

def remove_frequency(company_embeddings, embedding_dim):
    """
    Remove frequency dimension from company embeddings (assumed to be the top dimension)
    Parameters
    ----------
    company_embeddings : dict
        A dictionary mapping words to their embeddings
    embedding_dim : int
        The dimension of word vectors. Used to determine the number of components that go into PCA.
    Returns
    -------
    new_company_embeddings : dict
        A dictionary mapping words to their embeddings, where embeddings are demeaned and first PCA
        dimension hypothesized to represent frequency dimension is removed
    """
    vectors = np.array([v for k, v in company_embeddings.items()])
    miu = np.mean(vectors, axis=0)
    demeaned_vectors = vectors - miu
    pca = PCA(n_components = embedding_dim)
    pca.fit(demeaned_vectors)
    frequency_dim = pca.components_[0]
    new_company_embeddings = {k : drop(v-miu, frequency_dim) for k, v in company_embeddings.items()}
    return new_company_embeddings

def doPCA(words_start, words_end):
    """
    Performs PCA on differences between pairs of words and returns the first component
    Based on function doPCA in Bolukbasi et al. (2016) source code at https://github.com/tolga-b/debiaswe/blob/master/debiaswe/we.py
    Parameter
    ---------
    words_start : list
        List of hashed words at one end of interested dimension
    words_end: list
        List of hashed words at the other end of dimension
    Returns
    -------
    ndarray
        First component of PCA of differences between pairs of words
    """
    matrix = []
    for i in range(len(words_start)):
        center = (words_start[i] + words_end[i])/2
        matrix.append(words_end[i] - center)
        matrix.append(words_start[i] - center)
    matrix = np.array(matrix)
    # cannot have more components than the number of samples
    num_components = len(words_start)*2
    pca = PCA(n_components = num_components)
    pca.fit(matrix)
    return pca.components_[0]

def build_dimension(words_start, words_end):
    """
    This method builds a dimension defined by words at separate end of a dimension.
    Multiple methods exist in previous literature when building such a dimension.
    1) Kozlowski et al. (2019) averages across differences between different word pairs, noted to be interchangeable with averaging words on each side of the dimension and
    then taking the difference between averages. They are empirically verified to be identical.
    2) Bolukbasi et al. (2016) defines gender direction using a simple difference between man and woman in the corresponding tutorial. In the same tutorial, 
    racial direction is defined as difference between two clusters of words that are each sum of the embeddings of its corresponding dimensions
    normalized by the L2 norm. Wang et al. (2020) note that normalization is unnecessary. If unnormalized, this method should be equivalent to #3.
    3) Bolukbasi et al. (2016) defines gender direction also by taking the differences across multiple pairs, doing PCA on these differences, and 
    taking the first component as the gender direction.
    Parameter
    ---------
    words_start : list
        List of hashed words at the positive end of the dimension, where positive implies more likely to affect identification positively
    words_end: list
        List of hashed words at the other end of dimension
    Returns
    -------
    (mean_dim, pca_dimension) : 2-tuple of numpy vector
        Two vector that represents the dimension of interest calculated using method #1 and #3.
    """
    assert len(words_start) == len(words_end)
    differences = [(np.array(words_start[i]) - np.array(words_end[i])) for i in range(len(words_start)) if not np.isnan(words_start[i]).any() and not np.isnan(words_end[i]).any()]
    mean_dim = np.array(differences).mean(axis=0)
    pca_dim = doPCA(words_start, words_end)
    if project(words_start[0], pca_dim) < 0:
        # convention used in the current script is that words_start should represent the positive dimension
        pca_dim = pca_dim * -1
    return (mean_dim, pca_dim)

#ending here-------------------

Note the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Set the hyperparameters and the input and output directories

In [ ]:
#set the hyperparameters for Mittens
mittens_params = 0.1
window_size = 10 
embedding_dim = 50 
mincount = 50
max_iter = 50 #default
ling_thres = 0.8

#define flags to run for different cases
sampling_flag = False
north_american_flag = False

home_dir = "/zfs/projects/faculty/amirgo-identification/"
email_dir = os.path.join(home_dir, "email_data/")
mittens_dir = os.path.join(home_dir, "mittens")
utils_dir = os.path.join(mittens_dir, "utils")
output_dir = os.path.join(current_dir, "embeddings_high_prob_eng_{}_{}d_mincount{}_v2".format(str(ling_thres).replace(".", ""), embedding_dim, mincount))
#define a dict of hyperparameters
hyperparams = {'mittens_params': mittens_params, 'window_size':window_size, 'embedding_dim':embedding_dim, 'mincount':mincount, 'max_iter':max_iter, 'ling_thres':ling_thres, 'sampling':sampling_flag, 'north_american_only':north_american_flag}

#input files
email_file = os.path.join(email_dir, 'MessagesHashed.jsonl')
users_file = os.path.join(email_dir, 'Users.json')
activity_file = os.path.join(email_dir, 'Activities.json')
user_qualtrics_file = os.path.join(home_dir, "survey_hr_data", "UsersQualtrics.csv")

#trained embeddings from scratch using Glove
#the 50 in the company embeddings file name refers to the vector dimension and not mincount
company_embeddings_filename = "/zfs/projects/faculty/amirgo-identification/identification-Sarayu/staffing/training/vectors_high_prob_eng_08_50d.txt"
survey_filename = "/zfs/projects/faculty/amirgo-transfer/spacespace/spacespace/staffing/analyses_data/preprocessed_survey_hr.csv"
domain_hash = {
    'collabera.com':                     '509c8f6b1127bceefd418c023533d653', 
    'collaberainc.mail.onmicrosoft.com': 'ec5b67548b6ec06f1234d198efec741e', 
    'collaberainc.onmicrosoft.com':      '86160680578ee9258f097a67a5f25af9', 
    'collaberasp.com':                   '6bf3934d19f1acf5b9295b63e0e7f66e', 
    'g-c-i.com':                         '3444d1f7d5e46443080f2d069e41a10c'}
target_file = 'target_users.txt'

print(email_file)
print(users_file)
print(activity_file)
print(user_qualtrics_file)
print(company_embeddings_filename)
print('zfs/projects/faculty/amirgo-identification/identification-Sarayu/staffing/training/vectors_high_prob_eng_08_50d.txt')
print(output_dir)

In [ ]:
collabera_hashes = set([v for k, v in domain_hash.items()]) #create a set of the domain name hashes
user_sample_size = 8500
timekey_sample_size = 3500 

def load_target_users(target_file, regenerate=True, north_american_only=False): #output of this function is a file called 'target_users.txt'. These users have the survey data
    """
    Returns a set of User IDs, as used in anonymized emails, for whom survey-based identification data is available
    Parameters
    ----------
    target_file : str
        Full filepath to target users
    regenerate : bool, optional
        If true, regenerate target user file even if it already exists
    north_american_only : bool, optional
        If true, only return work IDs of employees working in North America
    """

#directly return the set of user id's present in the already generated target file 
    if not regenerate and os.path.exists(target_file):
        with open(target_file, "r") as file:
            userids = []
            for line in file:
                userids.append(line.strip()) # removing newline
            return set(userids)
    else: #regernerate the list of user IDs
        survey_df = pd.read_csv(survey_filename)
        if north_american_only:
            survey_df = survey_df.loc[survey_df['Work Country'] != "India"]
        uids_responded = survey_df.dropna(subset=['bergami_org_num'])['uid'].to_list()
        uids_responded += survey_df.dropna(subset=['mael_avg'])['uid'].to_list()
        uids_responded = list(set(uids_responded))
        user_qualtrics_df = pd.read_csv(user_qualtrics_file)
        user_qualtrics_df = user_qualtrics_df[user_qualtrics_df['UID'].isin(uids_responded)]
        emails_responded = user_qualtrics_df['Email']
        email2uid = {}
        with open(users_file, encoding='utf-8') as f:
            for line in f:
                user = json.loads(line)
                for e in user['Emails']:
                    email2uid[e] = user['UserId']
        userids = []
        for e in emails_responded:
            if e in email2uid.keys():
                userids.append(email2uid[e])
            else:
                print(e)
        with open(target_file, "w") as file:
            for u in userids:
                file.write(u+'\n')
        return set(userids)

def load_user_emails(target_users=None, test_mode=False): #this function is similar to the file 1_build_separate_corpus.py
    """
    Read emails and activities to return a dictionary that matches users to all their emails #what is meant by activities?
    Parameters
    ----------
    target_users : list
        If provided, only emails of individuals with user IDs included in this set are used for computation
        If None, all users are included for computation
    test_mode : bool, optional
        If testing, only load partial emails
    Returns
    -------
    uid2emails : dict
        A dictionary matching user ids to a list of emails of type dict
    """
    print('inside the load_users function')
    uid2emails = defaultdict(list)
    sid2activity = {}
    with open(activity_file, encoding='utf-8') as f:
        for line in f:
            activity = json.loads(line) #what does the activity file contain - some meta data, and we add more meta data to it below
            if target_users is None or activity['UserId'] in target_users:
                sid2activity[activity['MailSummarySid']] = activity
    target_sids = sid2activity.keys()
    print('making some progress...')
    with open(email_file, encoding='utf-8') as f:
        for i, line in enumerate(f): #read each email
            if test_mode and i > 100000:
                break
            if i % 10000000 == 0:
                print(i)
            email = json.loads(line) #read each email
            if email['sid'] in target_sids: #check if the email is in the target users list
                lang = email['l']
                if len(email['hb']) > 0 and lang[0] == "__label__en" and lang[1] > ling_thres:
                    # check internal or external or mixed
                    activity = sid2activity[email['sid']]
                    recipients = get_recipients(activity) #in utils.py, returns a set of recipients to the current email
                    pure_internal, pure_external = True, True
                    for r in recipients:
                        domain = r.split('@')[1] #get the part of teh recipients email address that is after the '@' symbol and categorize the domain
                        if pure_external and domain in collabera_hashes:
                            pure_external = False
                        elif pure_internal and domain not in collabera_hashes:
                            pure_internal = False
                    if pure_external:
                        activity['email_type'] = 'ext'
                    elif pure_internal:
                        activity['email_type'] = 'int'
                    elif not pure_external and not pure_internal:
                        activity['email_type'] = 'mixed'
                    elif pure_external and pure_internal:
                        activity['email_type'] = 'self'
                    # combine both JSONs into one
                    activity['hb'] = email['hb']
                    uid2emails[activity['UserId']].append(activity)
    return uid2emails

def process_user(i, num_users, uid, emails, email_type, timekeys=None, sampling=False):
    """
    Workhorse function for training embedding spaces for each individual. (by training here we mean training with a warm start - finetuning mittens on an individual level and 
    Parameters
    ----------
    i : int
        Index of current user used to keep track of progress
    num_users : int
        Total number of users used to keep track of progress
    uid : str
        User ID of current user
    emails : list of dict
        A list of emails converted from JSON formats that include both content and meta-data
    email_type : str
        Specifies which types of emails to include when building embeddings
    timekeys : list or None, optional
        If not None, only training embeddings for specified time periods
    sampling : bool, optionanl
        If sampling, restrict the number of emails at the user level and at the time key level to a fixed number
    """
    if i % 50 == 0:
        sys.stderr.write("\nProcessing \t%d/%d - user '%s' %s emails, at %s.\n" % (i, num_users, uid, email_type, datetime.now()))
    user_embedding_filename = os.path.join(output_dir, "{}_{}.txt".format(uid, email_type))
    if sampling:
        emails = random.choice(emails, user_sample_size)
    #finetune mittens
    if not os.path.exists(user_embedding_filename): #if the embedding file has not been already created in the output directory
        #first, create the cooccurance matrix
        X = build_weighted_matrix(emails, mincount=mincount, window_size=window_size, email_type=email_type) #utils.py
        if X.empty:
            return
        #now call mittens
        mittens = Mittens(n=embedding_dim, max_iter=max_iter, mittens=mittens_params)
        mittens = mittens.fit(
            X.values, #pass the cooccurrance matrix
            vocab=list(X.index), #the vocabulary itself, the axes are basically words
            initial_embedding_dict=company_embeddings) #company_embeddings generated form Glove training
        mittens_df = pd.DataFrame(mittens, index=X.index)
        if not mittens_df.empty: #store the output embeddings in this file
            output_embeddings(mittens_df, filename=user_embedding_filename)
    
    user_embeddings = glove2dict(user_embedding_filename) #utils, now the user_embeddings variable has mapping of every trained word in the vocab and its embedding

    sliced_usr_corpus = slice_user_corpus(emails, "all") #utils.py, assign a year and a quarter
    if timekeys is None:
        timekeys = sliced_usr_corpus.keys()
    #further finetune for each year and quarter
    for timekey in timekeys:
        emails = sliced_usr_corpus[timekey] #obtain the set of email in that year and quarter and half year
        if sampling: #sample if required
            emails = random.choice(emails, timekey_sample_size)
        
        user_embedding_time_filename = os.path.join(output_dir, "{}_{}_{}.txt".format(uid, timekey, email_type)) #user level embeddings for each year and quarter
        if not os.path.exists(user_embedding_time_filename):            
            X = build_weighted_matrix(emails, mincount=mincount, window_size=window_size, email_type=email_type)
            if X.empty:
                continue
            mittens = Mittens(n=embedding_dim, max_iter=max_iter, mittens=mittens_params)
            if not user_embeddings:
                sys.stderr.write("\n%s does not have corresponding user embeddings with timekey %s.\n" % (usr, timekey))
            mittens = mittens.fit(
                X.values,
                vocab=list(X.index),
                initial_embedding_dict=user_embeddings)
            mittens_df = pd.DataFrame(mittens, index=X.index)
            if not mittens_df.empty:
                output_embeddings(mittens_df, filename=user_embedding_time_filename) #here the finetuned user embeddings are supplied
    return

def process_users(uid2emails, test_mode, email_types=['both'], timekeys=None, sampling=False):
    """
    Processes emails of each user in parallel
    Parameter
    ---------
    uid2emails : dict
        A dictionary matching user IDs to a list of emails converted from JSON to dictionaries
    test_mode : bool
        If in test_mode, only process num_users_to_test users
    email_types : list, optional
        A list of email_types to process, either internal, external, or both
    timekeys : list, optional
        Timekeys that indicate time-specific embeddings to be computed
    """
    num_users = len(uid2emails)
    if test_mode:
        keys = random.choice(list(uid2emails), num_users_to_test)
        uid2emails = {k : uid2emails[k] for k in keys}
        num_users = num_users_to_test
    sys.stderr.write('Processing %d users in parallel at %s.\n' % (num_users, str(datetime.now())))

    pool = multiprocessing.Pool(processes = num_cores)
    results = [pool.apply_async(process_user, args=(i, num_users, uid, uid2emails[uid], email_type, timekeys, sampling,)) for i, uid in enumerate(uid2emails) for email_type in email_types]
    for r in results:
        r.get()
    pool.close()
    pool.join()
    return

In [ ]:
starttime = datetime.now()
for d in [mittens_dir, utils_dir, output_dir]:
    if not os.path.exists(d):
        os.mkdir(d)

with open(os.path.join(output_dir, 'hyperparams.txt'), 'w') as f:
    for k, v in hyperparams.items():
        f.write(k+' = '+str(v)+'\n')

sys.stderr.write("Loading target_users at {}.\n".format(str(datetime.now())))
#generate target users from north america
target_users = load_target_users(target_file, regenerate=True, north_american_only=north_american_flag)
    
sys.stderr.write("Loading all emails and activities for {} users at {}.\n".format(str(len(target_users)), str(datetime.now())))

# Set target_users to None when all users need to be computed
test_mode = False
uid2emails = load_user_emails(target_users, test_mode)
print('reached here')
company_embeddings = glove2dict(company_embeddings_filename) 

sys.stderr.write("Processing all users at {}.\n".format(str(datetime.now())))
# Remove these arguments to compute all emails
process_users(uid2emails, test_mode, email_types=['internal'], sampling=sampling_flag)

sys.stderr.write("\n\nFinished processing at {}, with a duration of {}\n".format(str(datetime.now()), str(datetime.now()-starttime)))
#ending here-------------------